In [11]:
import pandas as pd
import os 

os.environ["KERAS_BACKEND"] = "plaidml.keras.backend"

SQC_LEN=60 #la longueur d'une séquence est d'une minute 
FUTURE_PERIOD_PRED = 3 #Prédire le prix aprés 3 min 
FUTURE_RATIO_PRED = "ETH-USD"



main_df = pd.DataFrame() # begin empty
fichiers=["BTC-USD", "LTC-USD", "BCH-USD", "ETH-USD"] 

for file in fichiers: 
    print(file)
    
    dataset ="crypto_data/crypto_data/{}.csv".format(file)  # get the full path to the file.
    ?pd.read_csv
    df=pd.read_csv(dataset,names=['time', 'low', 'high', 'open', 'close', 'volume'])
    # rename volume and close 
    df.rename(columns={"close": "{}_close".format(file), "volume": "{}_volume".format(file)}, inplace=True)   
    #print(df.head())
    df.set_index("time", inplace=True)  # set time as index so we can join them on this shared time
    df = df[["{}_close".format(file), "{}_volume".format(file)]]  # ignorer  tous sauf le volume et the close 

    if len(main_df)==0: 
        main_df=df
    else : 
        main_df=main_df.join(df)
        
        
main_df.fillna(method="ffill", inplace=True) #si il ya des valeurs flou  utiliser s'eux d'auparavant ! 
main_df.dropna(inplace=True) #pour supprimer les valeurs manquantes


main_df.tail(10)

BTC-USD
LTC-USD
BCH-USD
ETH-USD


,BTC-USD_close,BTC-USD_volume,LTC-USD_close,LTC-USD_volume,BCH-USD_close,BCH-USD_volume,ETH-USD_close,ETH-USD_volume
time,,,,,,,,
1535214660,6707.799805,1.780853,58.000000,51.901546,531.479980,1.182528,279.089996,5.992979
1535214720,6708.100098,1.401337,58.009998,31.331142,531.469971,0.018900,279.299988,12.388200
1535214780,6708.379883,0.975295,58.009998,14.458084,531.479980,0.100199,279.299988,6.183691
1535214840,6710.089844,1.293573,58.009998,93.464951,531.479980,0.044015,279.290009,4.150405
1535214900,6712.990234,2.330975,58.020000,0.823356,531.469971,1.761348,279.299988,5.566861
1535214960,6713.140137,0.769891,58.020000,6.434783,531.479980,1.208560,279.359985,11.280577
1535215020,6714.520020,1.002652,58.009998,7.301921,531.479980,0.016868,279.359985,8.790519
1535215080,6714.520020,1.021925,58.020000,23.802017,531.469971,0.013854,279.369995,1.311763
1535215140,6715.000000,3.645508,58.020000,6.953497,531.479980,0.016900,279.660004,11.752819


Maintenant il nous faut une fonction  qui nous dit si c 'est un achat ou une vente 
pour identifier les classes 

In [15]:
def classify(future,current):
    if float(future) > float(current):
        return 1 #si 1 Achat sinon vente 
    else : 
        return 0 

#définissant maintenant notre colonne future ! 
print(main_df.tail(10))
main_df["future"]=main_df["{}_close".format(FUTURE_RATIO_PRED)].shift(-FUTURE_PERIOD_PRED)#shift vas faire un décalage de 3 ligne dans la colonne close

#mappons maintenant notre fonction sur la table en utilisant map qui prends en arguments  : la fonction puis ses parametres
#ensuites le resultat doit etre une list en utilisant List()

main_df["Cible"]=list(map(classify,main_df["future"],main_df["{}_close".format(FUTURE_RATIO_PRED)])) 
print(main_df[["{}_close".format(FUTURE_RATIO_PRED),"future","Cible"]].tail(10))


            BTC-USD_close  BTC-USD_volume  LTC-USD_close  LTC-USD_volume  \
time                                                                       
1534643040    6371.009766        1.539134      56.889999       57.140900   
1534643100    6371.009766        0.765422      56.919998        0.157505   
1534643160    6371.009766        1.459322      56.919998        0.157505   
1534643220    6370.009766        0.281895      56.919998        8.008989   
1534643280    6370.000000        2.610539      56.910000       13.368497   
1534643340    6370.009766        0.576384      56.910000       43.449566   
1534643400    6370.000000        2.011349      56.919998        6.114026   
1534643460    6365.200195        4.101097      56.910000        3.585331   
1534643520    6360.160156        8.866956      56.919998       10.954050   
1534643580    6360.160156        2.431280      56.919998        7.788442   

            BCH-USD_close  BCH-USD_volume  ETH-USD_close  ETH-USD_volume  \
time       

C:\Users\TRIUMPHAL\AppData\Local\conda\conda\envs\TensorProject\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
C:\Users\TRIUMPHAL\AppData\Local\conda\conda\envs\TensorProject\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


il y a un problem ! vu que nos données sont different les montant surtt on doit les normaliser ! et donc maintenant il est temps de les normaliser alons y !
il faut aussi les rendre equilibré ! 
mais avant nous devant séparé notre données au lieu de les mélanger car sa n'aura aucun sens ! 


In [14]:
#Séparation test et train data !
#trie croissant du temps ! 
times= sorted(main_df.index.values)#values nous donne une liste 
last_5pot=times[-int(0.05*len(times))]#last 5% du temps 

#séparation! 
validation_main_df=main_df[(main_df.index >= last_5pot)]#5%
main_df= main_df[(main_df.index < last_5pot)]#majorité 95% de notre data 


print("Training data ")
print(main_df[["{}_close".format(FUTURE_RATIO_PRED),"future","Cible"]].head(10))
print("Validation data ")
print(validation_main_df[["{}_close".format(FUTURE_RATIO_PRED),"future","Cible"]].head(10))



Training data 
            ETH-USD_close     future  Cible
time                                       
1528968720      486.01001  486.00000      0
1528968780      486.00000  486.00000      0
1528968840      485.75000  485.98999      1
1528968900      486.00000  485.98999      0
1528968960      486.00000  485.98999      0
1528969020      485.98999  485.98999      0
1528969080      485.98999  486.00000      1
1528969140      485.98999  486.00000      1
1528969200      485.98999  486.00000      1
1528969260      486.00000  486.00000      0
Validation data 
            ETH-USD_close      future  Cible
time                                        
1534643640     291.309998  291.559998      1
1534643700     291.309998  291.559998      1
1534643760     290.440002  291.570007      1
1534643820     291.559998  291.359985      0
1534643880     291.559998  291.519989      0
1534643940     291.570007  291.429993      0
1534644000     291.359985  291.429993      1
1534644060     291.519989  291.4299

In [4]:
#maint nous allons pouvoir faire  : la mise en échelle , la normalisation ,
# de  nos données avce la fonction modify_df

#sklearn est une librairie de machine learning qui propose des regressions , clustering ...
from sklearn import preprocessing
from collections import deque #deque est une fille doublement chainé avec une taille fixe ! qui peut etre donner avec le parametre maxlen 
import numpy as np 
import random 


def modify_df(df):
    #elimination de la colonne future  vu qu'il y a rien à modifier dans celle ci 
    df=df.drop("future",1)#1 veux dire colonne 
    for col in df.columns : 
        if col!="Cible":
            #on vas scaller tous sauf la Cible qui est soit 1 soit 0
            #Percentage change between the current and a prior element.
            df[col] = df[col].pct_change()  # pct change "normalizes" the different currencies (each crypto coin has vastly diff values, we're really more interested in the other coin's movements)
            df.dropna(inplace=True)  # remove the nas created by pct_change
            df[col] = preprocessing.scale(df[col].values)  # scale between 0 and 1 Centrer à la moyenne et composante par composante à la variance d'unité. pour les normaliser ! 
    
    df.dropna(inplace=True)  # cleanup again... jic. Those nasty NaNs love to creep in.
    #print(df)
    #Créant maintenant nos séquences ! 
    sequential_data= []
    prev_days=deque(maxlen=SQC_LEN)#ceci veut dire que  la taillle amx de notre deque est 60 ! et donc les valeurs ancienne sont supprimer ! 
    
    for i in df.values:  # iterate over the values ( une liste de liste )
        prev_days.append([n for n in i[:-1]])  # store all but the target 
        #pour chaque élement de df.values tous les valeurs apar le dernier  !  -1 c est le dernier ! Cible ! 
        
        if len(prev_days) == SQC_LEN:  # make sure we have 60 secondes!
            sequential_data.append([np.array(prev_days), i[-1]])  # i[-1] la cible de notre élement 

    random.shuffle(sequential_data)  # shuffle for good measure.
    
    #balance the sequential data ! 
    buys = []  # list that will store our buy sequences and targets
    sells = []  # list that will store our sell sequences and targets

    for seq, target in sequential_data:  #parcour de la séquence pour séparer les achats et vente 
        if target == 0:  # if it's a "not buy"
            sells.append([seq, target])  # append to sells list
        elif target == 1:  # otherwise if the target is a 1...
            buys.append([seq, target])  # it's a buy!

    random.shuffle(buys)  # shuffle the buys
    random.shuffle(sells)  # shuffle the sells!

    lower = min(len(buys), len(sells))  # what's the shorter length?
    #balancer les données ! 

    buys = buys[:lower]  # make sure both lists are only up to the shortest length.
    sells = sells[:lower]  # make sure both lists are only up to the shortest length.

    sequential_data = buys+sells  # rassembler les 2 
    random.shuffle(sequential_data)  # another shuffle, so the model doesn't get confused with all 1 class then the other.
    #séparer the x_test and y_test 
    X = []
    y = []
    #Séparer les inputs et outputs 

    for seq, target in sequential_data:  # going over our new sequential data
        X.append(seq)  # X is the sequences
        y.append(target)  # y is the targets/labels (buys vs sell/notbuy)

    return np.array(X), y  # return X and y...and make X a numpy array!





In [ ]:
import tensorflow as tf 
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Dropout,Dense,LSTM, CuDNNLSTM, BatchNormalization
from tensorflow.keras.callbacks import TensorBoard,ModelCheckpoint#ModelCheckpoint permet de sauvgarder des 
from tensorflow.keras.optimizers import RMSprop
#points importants et les suivre par exemple le max de la valeur de accuracy 
import time 

SQC_LEN=60 #la longueur d'une séquence est d'une minute 
FUTURE_PERIOD_PRED = 3 #Prédire le prix aprés 3 min 
FUTURE_RATIO_PRED = "ETH-USD"

x_train,y_train=modify_df(main_df)
x_val,y_val = modify_df(validation_main_df)


#quelque constantes utiles 
#le nombre d'iterations : 
EPOCHS=6
#BATCH SIZE 
BATCH_SIZE=64
#NAME  for our tensorBoard et aussi pour l'enrengister aprés 
NAME = "{}-SEQ-{}-PRED-{}".format(FUTURE_RATIO_PRED,SQC_LEN,FUTURE_PERIOD_PRED,int(time.time()))  # a unique name for the model 

#définition des couches : 

model=Sequential()

couche_1=LSTM(128,input_shape=x_train.shape[1:],activation='tanh',return_sequences=True)
model.add(couche_1)

couche_2=Dropout(0.2)
model.add(couche_2)

couche_3=BatchNormalization()
model.add(couche_3)

couche_4=LSTM(64,activation='tanh',return_sequences=True)
model.add(couche_4)

couche_5=Dropout(0.1)
model.add(couche_5)

couche_12=Dense(2, activation='softmax')
model.add(couche_12)


#compile  !
opt=RMSprop(lr=0.001)
'''tf.keras.optimizers.Adam(lr=0.001, decay=1e-6)
couche_10=Dense(32,activation='relu')
model.add(couche_10)

couche_11=Dropout(0.2)
model.add(couche_11)

gvs = opt.compute_gradients(cost)
capped_gvs = [(tf.clip_by_value(grad, -1., 1.), var) for grad, var in gvs]
train_op = opt.apply_gradients(capped_gvs)
opt.apply_gradients(capped_gvs)'''

matrics=['accuracy']
loss='sparse_categorical_crossentropy'
              #'sparse_categorical_crossentropy'
model.compile(loss=loss,
             optimizer=opt,
             metrics=['accuracy']
             )

#pour le tensorboard  : 
tensorboard=TensorBoard(log_dir="logs/{}".format(NAME))

#pour le ModelCheckpoint
              
filepath = "RNN_Final-{epoch:02d}-{val_acc:.3f}"  # unique file name that will include the epoch and the validation acc for that epoch
checkpoint = ModelCheckpoint("models/{}.model".format(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')) # saves only the best ones
#let's train the data ! 

      

Dans un premier temps j ai executer le modele avec les caractéristiques suivantes : 60-SEQ-3-PRED-1550310126
    EPOCHs= 10 
    BATCH=64 
    optimizer = adam ! 
    3 couches LSTM de 128 noeuds chaqunne avec une fonction d'activation = relu 
    1 couche dense 
    loss='sparse_categorical_crossentropy'
2éme : 60-SEQ-3-PRED-1550325231
   Epochs=5 
   BATCH=32
   optimisier = adam 
   3 couches LSTM avec tan h comme activation et 128 64 16 nombre de noeuds 
   loss='sparse_categorical_crossentropy'
   1 seule couche dense
3éme : 60-SEQ-3-PRED-1550332261
 Epochs=5 
   BATCH=32
   optimisier = adam 
   3 couches LSTM avec tan h comme activation et 128 64 16 nombre de noeuds 
   loss='sparse_categorical_crossentropy'
   2  couches dense
4éme : c 'est la 3 éme avec epochs=12 avec dérniére lstm 32 noeud au lieu de 16 

mazal hadi !5éme : 
Epochs=5 
   BATCH=32
   optimisier = adam 
   3 couches LSTM avec tan h comme activation et 128  nombre de noeuds 
   loss=binary-crossentropy
   1 couche dense

In [ ]:
model.fit(
    x_train, y_train,
    batch_size=BATCH_SIZE,
    epochs=EPOCHS,
    validation_data=(x_val, y_val)
 #  , callbacks=[tensorboard,checkpoint]
)            
        

In [ ]:
# Score model
score = model.evaluate(x_val,y_val, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])
# Save model
model.save("models/{}".format(NAME))

In [ ]:
a = [ i for i in range(1,4)]
a

In [ ]:
import numpy as np
np.expand_dims(a, axis=1)


In [ ]:
a=[[1,2],[2,3],[1,3,8],[4,9]]
a


In [ ]:
a[:-1]

In [ ]:
import numpy as ny


In [ ]:
z=np.array([[1,2,8],[2,3,5]])

In [ ]:
z[0]

In [ ]:
z[0:]

In [ ]:
p=np.random.randint(12,45,size=8)
p=np.array([p,z])
p

In [ ]:
z

In [ ]:
z[::-1,:]

In [ ]:
z[:,::-1]


In [ ]:
z[::2,::2]

In [ ]:
print(np.argmax(z))
print(np.max(z))

In [ ]:
z

In [ ]:
z=np.append(z,[1,3,5,9])

In [ ]:
z=np.array([1,2,3])
y=np.array([9,0,8])
z@y

In [ ]:
from matplotlib import pyplot as plt
x= [1,2,4]
y=[4,8,9]
plt.plot(x,y,'go')